In [1]:
!ls SDOHO/

PACO_V02.owl           TEO.owl                catalog-v001.xml
SDoHO_1005.owl         TEO_1.0.0_20200119.owl
SDoHO_1005.properties  bu_SDoHO_1005.owl


In [17]:
import json

json_path = "SDoHO/SDOHO.jsonld"
with open(json_path) as fh:
    ld = json.load(fh)

In [20]:
ld[0]

{'@id': '_:genid1',
 '@type': ['http://www.w3.org/2002/07/owl#Class'],
 'http://www.w3.org/2002/07/owl#unionOf': [{'@list': [{'@id': 'https://sbmi.uth.edu/bsdi/SDoHO#075'},
    {'@id': 'https://sbmi.uth.edu/bsdi/SDoHO#314'}]}]}

In [22]:
ld[1]

{'@id': '_:genid10',
 '@type': ['http://www.w3.org/2002/07/owl#Restriction'],
 'http://www.w3.org/2002/07/owl#onProperty': [{'@id': 'https://sbmi.uth.edu/bsdi/SDoH#has_measure'}],
 'http://www.w3.org/2002/07/owl#someValuesFrom': [{'@id': 'https://sbmi.uth.edu/bsdi/SDoHO#008'}]}

In [28]:
len(ld)

566

In [29]:
ld[-1]

{'@id': 'https://sbmi.uth.edu/ontology/TEO.owl#Time',
 'http://www.w3.org/2000/01/rdf-schema#subClassOf': [{'@id': 'https://sbmi.uth.edu/bsdi/SDoHO#384'}]}

In [32]:
for entry in ld:
    if "http://www.w3.org/2000/01/rdf-schema#subClassOf" in entry:
        print(entry)
        break

{'@id': 'http://www.semanticweb.org/hyk038/ontologies/2018/7/untitled-ontology-17#PACO', '@type': ['http://www.w3.org/2002/07/owl#Class'], 'http://www.w3.org/2000/01/rdf-schema#subClassOf': [{'@id': 'https://sbmi.uth.edu/bsdi/SDoHO#383'}]}


In [33]:
entry

{'@id': 'http://www.semanticweb.org/hyk038/ontologies/2018/7/untitled-ontology-17#PACO',
 '@type': ['http://www.w3.org/2002/07/owl#Class'],
 'http://www.w3.org/2000/01/rdf-schema#subClassOf': [{'@id': 'https://sbmi.uth.edu/bsdi/SDoHO#383'}]}

In [2]:
from rdflib import Graph

In [3]:
path = "SDoHO/SDoHO_1005.owl"

In [4]:
# # Create a new RDF graph
# graph = Graph()

# # Load the OWL file into the graph
# graph.parse(path, format='xml')

In [5]:
import pandas as pd
import owlready2

In [35]:
# # Create a new world
# world = World()

# # Load the ontology file
# path = "SDoHO/SDoHO_1005.owl"
# ontology = world.get_ontology('file://' + path).load()

In [11]:
from owlready2 import *


class SparqlQueries:
    def __init__(self, path, 
                 base="http://www.semanticweb.org/hyk038/ontologies/2018/7/untitled-ontology-17/"):
        self.base = base
        my_world = World()
        ont = Ontology(my_world, base)
#         ont.set_imported_ontologies("./TEO.owl")
        with open(path, "rb") as fh:
            ont.load(only_local=True, fileobj=fh)
#         my_world.get_ontology(f"file://{path}").load() #path to the owl file is given here
        sync_reasoner(my_world)  #reasoner is started and synchronized here
        self.graph = my_world.as_rdflib_graph()

    def search(self, query=None):
        #Search query is given here
        #Base URL of your ontology has to be given here
        if query is None:
            query = f"base <{self.base}> " \
                    """SELECT 
                        ?s ?p ?o 
                    WHERE { 
                        ?s ?p ?o
                    }"""

        #query is being run
        resultsList = self.graph.query(query)

        #creating json object
        response = []
        for item in resultsList:
#             print(item)
            s = str(item['s'].toPython())
            s = re.sub(r'.*#',"",s)
            
            p = item['p']
            if p is not None:
                p = str(p.toPython())
                p = re.sub(r'.*#', "", p)

            o = str(item['o'].toPython())
            o = re.sub(r'.*#', "", o)
            response.append({"s":s, "relation":p, "o":o})
        return response

In [34]:
path = "SDoHO/SDoHO_1005.owl"
# base = "http://www.semanticweb.org/navyarenjith/ontologies/2022/6/untitled-ontology-67"
owl_ = SparqlQueries(path,
                     base="file:///Users/dlituiev/repos/ontologies/SDoHO/"
#                      base = "http://www.semanticweb.org/hyk038/ontologies/2018/7/untitled-ontology-17#"
                    )

OwlReadyOntologyParsingError: Cannot download 'http://www.semanticweb.org/hyk038/ontologies/2018/7/untitled-ontology-17/'!

In [ ]:
owl_.search()

In [ ]:
owl_.graph

In [118]:
res = list(owl_.graph.all_nodes())

In [121]:
ont = res[0]

In [122]:
ont.

rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Ontology')

In [70]:
query="""SELECT 
    ?s ?p ?o 
WHERE { 
    ?s rdfs:label ?o .
}"""

labels = [(res["s"], res["o"]) for res in owl_.search(query)]

In [71]:
labels = pd.DataFrame(labels, columns=["id", "label"])

In [72]:
labels

,id,label


In [54]:
query="""SELECT 
    ?s ?p ?o 
WHERE { 
    ?s rdfs:subClassOf ?o .
}"""

parents = [(res["s"], res["o"]) for res in owl_.search(query)]

In [58]:
labels = pd.DataFrame(labels, columns=["id", "label"])

In [56]:
parents = pd.DataFrame(parents, columns=["child_id", "parent_id"])

In [57]:
pd.merge(
    pd.merge(parents, labels, left_on="child_id", right_on="id").drop(columns="id").rename(columns={"label":"child"}),
    labels, left_on = "parent_id", right_on = "id").drop(columns="id").rename(columns={"label":"parent"})

,child_id,parent_id,child,parent


In [ ]:
query="""SELECT 
    ?s ?p ?o 
WHERE { 
    ?s rdfs:label ?o .
}"""

owl_.search(query)

In [73]:
nodes = owl_.graph.all_nodes()
nodes = list(nodes)

In [74]:
nodes[:10]

[rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Ontology'),
 rdflib.term.URIRef('http://inferrences'),
 rdflib.term.URIRef('http://anonymous')]

In [43]:
node = nodes[1]
node

rdflib.term.Literal('Inability to pay for medication', lang='en')

In [50]:
node.n3()

'"Inability to pay for medication"@en'